In [1]:
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data
X = np.load("X_train.npy")      # replace with your file names
y = np.load("Y_train.npy")

print("Shape:", X.shape)
print("Labels shape:", y.shape)
print("Classes:", np.unique(y))
print("Class distribution:")
unique, counts = np.unique(y, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  Class {u}: {c} samples")
    
# Convert to DataFrame for easy EDA
df = pd.DataFrame(X)

df = df.drop(columns=[31, 33, 56, 57, 58, 59, 60, 61])


# Check for missing values
print("\nMissing values per column (first 10):")
print(df.isna().sum().head(10))

# Basic statistics
print("\nBasic statistics:")
print(df.describe().transpose().head(10))

# Look for constant or near-constant features
print("\nZero-variance features:")
zero_var = df.nunique()
print(zero_var[zero_var <= 1])




Shape: (2059408, 93)
Labels shape: (2059408,)
Classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Class distribution:
  Class 0: 1718707 samples
  Class 1: 1563 samples
  Class 2: 102411 samples
  Class 3: 8229 samples
  Class 4: 138279 samples
  Class 5: 4182 samples
  Class 6: 4308 samples
  Class 7: 4746 samples
  Class 8: 9 samples
  Class 9: 29 samples
  Class 10: 72655 samples
  Class 11: 2575 samples
  Class 12: 1176 samples
  Class 13: 17 samples
  Class 14: 522 samples

Missing values per column (first 10):
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

Basic statistics:
       count          mean  std       min       25%       50%       75%  \
0  2059408.0 -2.516941e-17  1.0 -0.452141 -0.449333 -0.447903 -0.428671   
1  2059408.0  1.918327e-17  1.0 -0.464741 -0.464735 -0.463344 -0.315312   
2  2059408.0 -1.380091e-18  1.0 -0.011537 -0.010301 -0.010301 -0.006594   
3  2059408.0  1.138575e-19  1.0 -0.010837 -0.009908 -0.008980 -0.006194 

In [2]:
import numpy as np
import pandas as pd

X = np.load("X_train.npy")
y = np.load("Y_train.npy")


In [3]:
with open("feature_names.txt", "r") as f:
    feature_names = [line.strip() for line in f.readlines()]

df = pd.DataFrame(X, columns=feature_names)
labels = pd.Series(y, name="label")


In [4]:
variance = df.var()
zero_var_cols = variance[variance == 0].index.tolist()

df = df.drop(columns=zero_var_cols)


In [5]:
df = df.clip(-5, 5)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_clean = scaler.fit_transform(df)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_clean, labels, test_size=0.2, random_state=42, stratify=labels
)


In [7]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(y_train)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train
)

class_weight_dict = {cls: w for cls, w in zip(classes, class_weights)}
print(class_weight_dict)


{0: 0.07988208184693187, 1: 87.86805333333334, 2: 1.340612807023968, 3: 16.684652387462656, 4: 0.9928773100229308, 5: 32.82578202829249, 6: 31.873205649061713, 7: 28.926801861118427, 8: 15690.72380952381, 9: 4775.43768115942, 10: 1.889668065973895, 11: 53.317993527508094, 12: 116.72164364151612, 13: 7845.361904761905, 14: 262.76331738437}


In [8]:
from imblearn.combine import SMOTETomek

sm = SMOTETomek(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

rare_classes = [8, 9, 13]

y_resampled = y_resampled.replace(rare_classes, 99)   # label 99 = "RareAttack"


MemoryError: Unable to allocate 889. MiB for an array with shape (1371519, 85) and data type float64

In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# ----------------------------------------------------
# 1. MEMORY-MAP TRAIN/TEST FILES
# ----------------------------------------------------
X_train = np.load("X_train.npy", mmap_mode='r').astype(np.float32)
y_train = np.load("Y_train.npy", mmap_mode='r').copy() 
X_test = np.load("X_test.npy", mmap_mode='r').astype(np.float32)
y_test = np.load("Y_test.npy", mmap_mode='r').copy()

print("Original shapes:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)


Original shapes:
X_train: (2059408, 93) y_train: (2059408,)
X_test: (514853, 93) y_test: (514853,)


In [24]:
# ----------------------------------------------------
# 2. DROP ZERO-VARIANCE COLUMNS (SCAN COLUMN-WISE)
# ----------------------------------------------------
zero_var_cols = []
for col in range(X_train.shape[1]):
    if np.ptp(X_train[:, col]) == 0:  # ptp = max - min
        zero_var_cols.append(col)

print(f"Dropping {len(zero_var_cols)} zero-variance columns:", zero_var_cols)

X_train = np.delete(X_train, zero_var_cols, axis=1)
X_test = np.delete(X_test, zero_var_cols, axis=1)

# ----------------------------------------------------
# 3. MERGE EXTREMELY RARE CLASSES
# ----------------------------------------------------
rare_classes = [8, 9, 13]  # Heartbleed, Infiltration, Web Attack SQL inj
for cls in rare_classes:
    y_train[y_train == cls] = 99
    y_test[y_test == cls] = 99

# ----------------------------------------------------
# 4. STANDARDIZE FEATURES IN BATCHES
# ----------------------------------------------------
scaler = StandardScaler()

# Fit scaler on train
# If too large, can batch with partial_fit (here we do in one go for float32)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Scaling done.")
print("Final train shape:", X_train_scaled.shape)
print("Final test shape:", X_test_scaled.shape)

Dropping 8 zero-variance columns: [31, 33, 56, 57, 58, 59, 60, 61]
Scaling done.
Final train shape: (2059408, 85)
Final test shape: (514853, 85)


In [25]:
# ----------------------------------------------------
# 5. SAVE PREPROCESSED FILES
# ----------------------------------------------------
np.save("X_train_preprocessed.npy", X_train_scaled)
np.save("y_train_preprocessed.npy", y_train)
np.save("X_test_preprocessed.npy", X_test_scaled)
np.save("y_test_preprocessed.npy", y_test)

print("Preprocessing complete. Preprocessed files saved.")

Preprocessing complete. Preprocessed files saved.
